# Scrape available traffic data dates for each node

1.获取所有node的metadata https://gdottrafficdata.drakewell.com/dataserver/sites
2.爬取每个node记录了交通数据的日期

本项目采用模拟请求+抓包分析的方式，从 GDOT 官方交通数据平台批量抓取历史交通信息。具体做法如下：

抓包分析请求结构
通过浏览器开发者工具（Network 面板），定位网页向服务器发送的数据请求，获取目标 URL、请求方法（POST）、参数结构和请求头（headers）。

复刻请求逻辑
使用 Python 的 requests 库构造与浏览器一致的 POST 请求，包含动态生成的 User-Agent、Referer 和 Cookie，以模拟真实用户访问。

批量节点爬取
从 CSV 文件中读取所有待爬取的节点编号（cosit + id），并逐个发送请求获取对应数据。

多线程并发执行
使用 ThreadPoolExecutor 同时发送多个请求，大幅提升爬取效率。

反封锁措施

随机 User-Agent 避免指纹识别

请求间随机暂停 0.1~1 秒，模拟正常用户行为

跳过已下载数据，避免重复请求与封 IP 风险


# Metadata数据介绍

This is the site that has all the station metadata: https://gdottrafficdata.drakewell.com/dataserver/sites

把这个数据保存为gdot_sites.json。

有两种node，一种是长期记录数据的（gdot网站上表示为绿色三角，cosit没下划线），一种是一年只记录三天的（gdot网站上表示为小蓝圆点，cosit有下划线），先把这两种分开。

# 分离两种node的metadata

In [1]:
import json
import pandas as pd

# 读取 JSON 数据
with open("gdot_sites.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 提取所有的 cosit 数据
cosit_list = [entry["cosit"] for entry in data["data"].values() if "cosit" in entry]

# 统计包含 "_" 和不包含 "_" 的数量
cosit_with_underscore = [cosit for cosit in cosit_list if "_" in cosit]
cosit_without_underscore = [cosit for cosit in cosit_list if "_" not in cosit]

# 输出数量
print(f"包含下划线的 cosit 数量: {len(cosit_with_underscore)}")
print(f"不包含下划线的 cosit 数量: {len(cosit_without_underscore)}")


包含下划线的 cosit 数量: 26981
不包含下划线的 cosit 数量: 331


打印所有不同种类的node

In [2]:
import json

# 读取 JSON 文件
with open("gdot_sites.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 提取所有不同的 node 值
unique_nodes = set()

for entry in data.get("data", {}).values():  # 遍历 data["data"] 的值
    node_type = entry.get("node")
    if node_type:
        unique_nodes.add(node_type)

# 打印所有不同的 node 类型
print("所有不同的 node 类型:")
for node in sorted(unique_nodes):
    print(node)


所有不同的 node 类型:
GDOT_CCS
GDOT_PORTABLES


# 分别提取两种node的信息

##### 取出所有node=GDOT_CCS的cosit和id 即长期记录数据的（gdot网站上表示为绿色三角，cosit没下划线）点的信息

In [ ]:
import json
import pandas as pd

# dot metadata的JSON 文件名是 "gdot_sites.json"
with open("gdot_sites.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 提取所有符合条件的记录
results = []
for entry in data.get("data", {}).values():  # 遍历 data["data"] 的值
    if entry.get("node") == "GDOT_CCS":  # 筛选 node 为 GDOT_PORTABLES
        cosit = entry.get("cosit")
        node_id = entry.get("id")
        if cosit and node_id:  # 确保 cosit 和 id 存在
            results.append({"cosit": cosit, "id": node_id})

# 转换为 DataFrame
df = pd.DataFrame(results)

# 输出 DataFrame
print(df)

# 可选：保存到 CSV 文件
df.to_csv("cosit_and_id_for_ccs_dot.csv", index=False)

            cosit                                    id
0    000000010183  573292FF-2F4B-4721-92BB-B2278A482D0F
1    000000010185  73997343-DF95-41E3-9001-7A7B6F3748A7
2    000000030132  7A9637CD-2F9E-49CA-A3FF-0B9256F66E40
3    000000030138  F61A1A60-F714-4CC5-AEA1-2C68270317F4
4    000000050125  FE1BBC25-7210-4D7C-A2AD-193BE9120E6F
..            ...                                   ...
325  000002990007  48B5322B-12AB-4D90-8E52-0D8DC384DAB3
326  000002990309  CFD95487-F373-46A4-8414-A43E6405AC24
327  000003010196  D1B94F8B-DA2C-4F9F-BB89-AE4D8AA2F706
328  000003130101  8ADB98AB-C07B-486D-87FA-2039355EB38D
329  000003190167  CA8F5A30-D4D0-452B-A9E1-E24493D86C61

[330 rows x 2 columns]


##### 取出所有node=GDOT_PORTABLES的cosit和id 即一年只记录三天的（gdot网站上表示为小蓝圆点，cosit有下划线）的信息

In [ ]:
# 取出所有node=GDOT_PORTABLES的cosit和id 即小蓝圆点的信息
import json
import pandas as pd

# 假设你的 JSON 文件名是 "gdot_sites.json"
with open("gdot_sites.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 提取所有符合条件的记录
results = []
for entry in data.get("data", {}).values():  # 遍历 data["data"] 的值
    if entry.get("node") == "GDOT_PORTABLES":  # 筛选 node 为 GDOT_PORTABLES
        cosit = entry.get("cosit")
        node_id = entry.get("id")
        if cosit and node_id:  # 确保 cosit 和 id 存在
            results.append({"cosit": cosit, "id": node_id})

# 转换为 DataFrame
df = pd.DataFrame(results)

# 输出 DataFrame
print(df)

# 可选：保存到 CSV 文件
df.to_csv("cosit_and_id_for_blue_dot.csv", index=False)

              cosit                                    id
0      0000001_0101  A5C0D853-0D00-4C1A-88C4-81554113CF1C
1      0000001_0105  ADE89047-85EC-4C5B-9433-FC41EE009322
2      0000001_0107  E9DF8D0F-39C3-4B52-9FE5-39F9BD92FC86
3      0000001_0109  4915A6A5-A22D-4411-A2AA-2539D712A741
4      0000001_0112  9B32EB1E-D0EA-4AA2-B5BB-865CA8AD8A80
...             ...                                   ...
26977  0000321_8109  73B21091-0170-43D4-A4EC-01E2EB67141E
26978  0000321_8110  3FE21761-E794-43B8-97B4-7B60CEFF6803
26979  0000321_8111  A0175972-F929-4BEE-B0FA-353BB0B46D42
26980  0000321_8112  366FCAA9-8392-4BBA-8FA7-72C78DDA375F
26981  0000321_8113  4994CA42-8252-47D9-8FE1-58A992A385F2

[26982 rows x 2 columns]


亚特兰大边界内的CCS点

In [ ]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 读取 JSON 数据
with open("gdot_sites.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 读取亚特兰大的 GeoJSON 边界
atlanta_boundary = gpd.read_file("atlanta_boundary.geojson")

# 提取符合条件的记录
results = []
for entry in data.get("data", {}).values():
    if entry.get("node") == "GDOT_CCS":
        cosit = entry.get("cosit")
        node_id = entry.get("id")
        lat = entry.get("location", {}).get("lat")
        lng = entry.get("location", {}).get("lng")

        if cosit and "_" in cosit and lat and lng:
            point = Point(lng, lat)  # 创建点对象
            if atlanta_boundary.contains(point).any():  # 检查点是否在边界内
                results.append({"cosit": cosit, "id": node_id, "lat": lat, "lng": lng})

# 转换为 DataFrame
df = pd.DataFrame(results)

# 输出 DataFrame
print(df)

# 可选：保存到 CSV 文件
df.to_csv("CCS_filtered_cosit_and_id_in_atlanta.csv", index=False)

Empty DataFrame
Columns: []
Index: []


/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:350: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:350: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:354: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)


亚特兰大边界内的小蓝圆点

In [2]:
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 读取 JSON 数据
with open("gdot_sites.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 读取亚特兰大的 GeoJSON 边界
atlanta_boundary = gpd.read_file("atlanta_boundary.geojson")

# 提取符合条件的记录
results = []
for entry in data.get("data", {}).values():
    if entry.get("node") == "GDOT_PORTABLES":
        cosit = entry.get("cosit")
        node_id = entry.get("id")
        lat = entry.get("location", {}).get("lat")
        lng = entry.get("location", {}).get("lng")

        if cosit and "_" in cosit and lat and lng:
            point = Point(lng, lat)  # 创建点对象
            if atlanta_boundary.contains(point).any():  # 检查点是否在边界内
                results.append({"cosit": cosit, "id": node_id, "lat": lat, "lng": lng})

# 转换为 DataFrame
df = pd.DataFrame(results)

# 输出 DataFrame
print(df)

# 可选：保存到 CSV 文件
df.to_csv("filtered_cosit_and_id_in_atlanta.csv", index=False)


/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:350: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:350: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
/opt/anaconda3/lib/python3.9/site-packages/geopandas/io/file.py:354: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore", utc=True)


            cosit                                    id        lat        lng
0    0000089_3001  7586A93E-57CA-4696-B2BD-2B2748FF1E3E  33.780031 -84.308190
1    0000089_3003  F1CBF558-9E0C-43ED-BDA0-7817C3BADC16  33.789051 -84.297380
2    0000089_3005  3EF6347D-C1C2-4436-AA95-8C042297B715  33.774035 -84.335479
3    0000089_3007  6E9DE2D5-7411-4E14-B3F2-AB9E072B5CCE  33.772553 -84.317501
4    0000089_3009  EDD4D3FE-C2DC-4092-B71B-45BC329F1C56  33.774498 -84.305264
..            ...                                   ...        ...        ...
588  0000121_R883  4AF43658-5C4A-426E-B945-772F646E801F  33.696259 -84.411881
589  0000121_R884  42125878-EFA1-452F-BC2E-CF2F484C666A  33.696791 -84.411629
590  0000121_R885  4B85F907-62F9-4FED-9A35-1C1EE0642BC1  33.696847 -84.412122
591  0000121_R886  E87CF2D8-F053-41E8-905F-A164E6030518  33.795185 -84.388145
592  0000121_R998  B6BC9266-EAE7-4471-9886-931D022C6108  33.791260 -84.391010

[593 rows x 4 columns]


我们发现亚特兰大边界内的点比较少，所以不考虑从亚特兰大范围开始做，直接做整个佐治亚州的。

# 爬虫
分别爬取两种node的具体信息，关键是要取出是哪些天数有记录数据。

##### 爬取所有一年只记录三天的（gdot网站上表示为小蓝圆点，cosit有下划线）点的信息

bluedot数据1已经保存，直接从这里开始

In [2]:
# 加载 CSV 文件 bluedot数据
import pandas as pd
blue_dot = pd.read_csv("cosit_and_id_for_blue_dot.csv")

In [3]:
blue_dot

,cosit,id
0,0000001_0101,A5C0D853-0D00-4C1A-88C4-81554113CF1C
1,0000001_0105,ADE89047-85EC-4C5B-9433-FC41EE009322
2,0000001_0107,E9DF8D0F-39C3-4B52-9FE5-39F9BD92FC86
3,0000001_0109,4915A6A5-A22D-4411-A2AA-2539D712A741
4,0000001_0112,9B32EB1E-D0EA-4AA2-B5BB-865CA8AD8A80
...,...,...
26977,0000321_8109,73B21091-0170-43D4-A4EC-01E2EB67141E
26978,0000321_8110,3FE21761-E794-43B8-97B4-7B60CEFF6803
26979,0000321_8111,A0175972-F929-4BEE-B0FA-353BB0B46D42
26980,0000321_8112,366FCAA9-8392-4BBA-8FA7-72C78DDA375F


## 单个爬取示例

In [ ]:
import requests
import json

# 请求 URL
#gdots_sites.json-id
url = "https://gdottrafficdata.drakewell.com/dataserver/tabularCalendar/30F053C6-AD42-40CE-90A5-385A1BB3B5A4"#这里写上每个id

# 请求头
headers = {
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en,zh;q=0.9,zh-CN;q=0.8",
    "cache-control": "no-cache",
    "content-type": "application/json",
    "origin": "https://gdottrafficdata.drakewell.com",
    "referer": "https://gdottrafficdata.drakewell.com/calendar_alt.asp?node=GDOT_PORTABLES&cosit=0000095_0317",#这里写上每个cosit
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Mobile Safari/537.36",
    "x-requested-with": "XMLHttpRequest",
    "cookie": "_ga=GA1.1.1478648062.1737667423; ASPSESSIONIDAWDACTBQ=FCHPNCEALJEDEOIADIMLLKPF; _ga_02J23J4R19=GS1.1.1738639325.19.1.1738639607.0.0.0"
}

# 请求体 (根据实际内容调整)
payload = {
    "startDate": "2010-02-15",  # 示例参数
    "endDate": "2025-02-15"    # 示例参数
}

# 发送 POST 请求
response = requests.post(url, headers=headers, data=json.dumps(payload))

# 检查响应状态
if response.status_code == 200:
    # 如果响应成功，解析 JSON 数据
    data = response.json()
    print(json.dumps(data, indent=4))  # 美化输出
else:
    print(f"请求失败，状态码: {response.status_code}")

{
    "data": [
        {
            "day": 14655,
            "hasTrafficData": true,
            "dType": 3,
            "rawType": 1,
            "isFullyChecked": true,
            "isZeroFlow": false,
            "isPartialCoverage": true
        },
        {
            "day": 14656,
            "hasTrafficData": true,
            "dType": 3,
            "rawType": 1,
            "isFullyChecked": true,
            "isZeroFlow": false,
            "isPartialCoverage": false
        },
        {
            "day": 14657,
            "hasTrafficData": true,
            "dType": 3,
            "rawType": 1,
            "isFullyChecked": true,
            "isZeroFlow": false,
            "isPartialCoverage": true
        },
        {
            "day": 16161,
            "hasTrafficData": true,
            "dType": 1,
            "rawType": 1,
            "isFullyChecked": true,
            "isZeroFlow": false,
            "isPartialCoverage": true
        },
        {
            "

# 多个爬取

小蓝圆点日期 多个爬取

随机 User-Agent 头部，减少被封风险。

ThreadPoolExecutor 并行爬取，加速数据获取。

tqdm 进度条，实时监控进度。

避免重复爬取，已爬取数据直接跳过。

随机 0.1-1.0 秒间隔，降低封 IP 风险。

存储到 blue_dot_date_json/ 目录，每个 cosit_id.json 一个文件。

In [ ]:
import requests
import json
import pandas as pd
import os
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 读取 CSV 文件
blue_dot = pd.read_csv("cosit_and_id_for_blue_dot.csv")

# 确保存储文件夹存在
output_dir = "blue_dot_date_json"
os.makedirs(output_dir, exist_ok=True)

# 请求基础 URL
base_url = "https://gdottrafficdata.drakewell.com/dataserver/tabularCalendar/{}"

# 预定义多个 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Linux; Android 11; SM-G998B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
]

# 请求体
payload = {
    "startDate": "2000-01-01",
    "endDate": "2025-02-15"
}

# 进度条 & 统计信息
total_tasks = len(blue_dot)

# 爬取单个站点的函数
def fetch_data(cosit, node_id):
    """爬取指定 cosit 和 id 的数据"""
    # 生成存储文件名
    filename = os.path.join(output_dir, f"{cosit}_{node_id}.json")

    # 检查文件是否已存在，避免重复爬取
    if os.path.exists(filename):
        return f"已存在，跳过: {cosit} - {node_id}"

    # 随机选择 User-Agent
    user_agent = random.choice(USER_AGENTS)

    # 请求头
    headers = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en,zh;q=0.9,zh-CN;q=0.8",
        "cache-control": "no-cache",
        "content-type": "application/json",
        "origin": "https://gdottrafficdata.drakewell.com",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": user_agent,  # 使用随机 User-Agent
        "x-requested-with": "XMLHttpRequest",
        "cookie": "_ga=GA1.1.1478648062.1737667423; ASPSESSIONIDAWDACTBQ=FCHPNCEALJEDEOIADIMLLKPF; _ga_02J23J4R19=GS1.1.1738639325.19.1.1738639607.0.0.0"
    }

    # 更新 URL 和 Referer 头
    url = base_url.format(node_id)
    headers["referer"] = f"https://gdottrafficdata.drakewell.com/calendar_alt.asp?node=GDOT_PORTABLES&cosit={cosit}"

    try:
        # 发送请求
        response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=10)

        if response.status_code == 200:
            # 解析 JSON 数据
            data = response.json()
            result = {"cosit": cosit, "id": node_id, "data": data}

            # 保存数据
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(result, f, indent=4, ensure_ascii=False)

            return f"成功获取: {cosit} - {node_id}"

        else:
            return f"请求失败: {cosit} - {node_id}, 状态码: {response.status_code}"

    except requests.exceptions.RequestException as e:
        return f"请求出错: {cosit} - {node_id}, 错误: {e}"

    finally:
        # 随机暂停 0.1 - 1.0 秒，防止封 IP
        time.sleep(random.uniform(0.1, 1.0))


# **多线程爬取**
max_workers = 10  # 可调整并发数，根据网络情况设定

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 提交任务
    futures = {executor.submit(fetch_data, row["cosit"], row["id"]): (row["cosit"], row["id"]) for _, row in blue_dot.iterrows()}

    # 进度条
    with tqdm(total=total_tasks, desc="爬取进度") as pbar:
        for future in as_completed(futures):
            result = future.result()
            print(result)  # 打印爬取状态
            pbar.update(1)  # 更新进度条

print("所有数据爬取完成！")

爬取进度:   0%|          | 10/26982 [00:00<08:39, 51.94it/s]

成功获取: 0000001_0118 - D7AF5274-F0A2-4047-8279-1F90B4CCC9CE
成功获取: 0000001_0101 - A5C0D853-0D00-4C1A-88C4-81554113CF1C
成功获取: 0000001_0123 - 8C0C3F65-AAAB-4AD8-80B4-024C825DE19C
成功获取: 0000001_0114 - 1BFC7CDC-CBF2-43F4-BB93-3DEC56395073
成功获取: 0000001_0105 - ADE89047-85EC-4C5B-9433-FC41EE009322
成功获取: 0000001_0109 - 4915A6A5-A22D-4411-A2AA-2539D712A741
成功获取: 0000001_0125 - 78C05144-4B23-4D03-864F-31E2472B01CF
成功获取: 0000001_0107 - E9DF8D0F-39C3-4B52-9FE5-39F9BD92FC86
成功获取: 0000001_0121 - EF1B1A64-944F-46BD-9151-234814379B10
成功获取: 0000001_0112 - 9B32EB1E-D0EA-4AA2-B5BB-865CA8AD8A80
成功获取: 0000001_0127 - E985BE2D-624F-4474-920B-C34711626558
成功获取: 0000001_0129 - 51079F8B-259B-4675-9DCC-36673F879DCE
成功获取: 0000001_0136 - A16D4CAF-FF81-433D-B4A1-A752B027BAA3
成功获取: 0000001_0132 - 2ECB3593-DD5F-47C2-A8EC-275B51915AEA
成功获取: 0000001_0141 - 8FA0A18D-5A90-4CA2-B044-E788F884B4AB


爬取进度:   0%|          | 19/26982 [00:01<26:41, 16.84it/s]

成功获取: 0000001_0145 - 712EF486-8AE0-4C4C-8A39-FBED598F0DE2
成功获取: 0000001_0156 - 60EA259A-2AEB-4AC4-8FD5-55D247998B05
成功获取: 0000001_0143 - 7770D11F-DBCE-437E-898D-B80D5000A075
成功获取: 0000001_0163 - 72CCEE4E-10F6-4058-ADC7-1336BEB68CF4
成功获取: 0000001_0134 - 9625B08E-60CF-44E1-B8C6-4DE7A1675703


爬取进度:   0%|          | 23/26982 [00:01<24:56, 18.01it/s]

...

成功获取: 0000321_8108 - 83C51C85-CED1-478B-88B8-4D7C057DAF72
成功获取: 0000321_8112 - 366FCAA9-8392-4BBA-8FA7-72C78DDA375F
成功获取: 0000321_8111 - A0175972-F929-4BEE-B0FA-353BB0B46D42


爬取进度: 100%|█████████▉| 26980/26982 [37:29<00:00,  9.61it/s]

成功获取: 0000321_8109 - 73B21091-0170-43D4-A4EC-01E2EB67141E
成功获取: 0000321_8110 - 3FE21761-E794-43B8-97B4-7B60CEFF6803


爬取进度: 100%|██████████| 26982/26982 [37:29<00:00, 12.00it/s]

成功获取: 0000321_8113 - 4994CA42-8252-47D9-8FE1-58A992A385F2
所有数据爬取完成！


#### 处理爬取出来的json文件

In [5]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta

# 设定 JSON 文件夹路径
json_folder = "blue_dot_date_json"

# 设定起始日期（1970-01-01）
base_date = datetime(1970, 1, 1)

# 结果存储列表
results = []

# 遍历 JSON 文件夹
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(json_folder, filename)

        try:
            # 读取 JSON 文件
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # 提取 cosit 和 id
            cosit = data.get("cosit", "unknown")
            node_id = data.get("id", "unknown")

            # 提取天数并转换为日期格式
            if "data" in data and "data" in data["data"]:
                for entry in data["data"]["data"]:
                    day_number = entry.get("day")
                    if day_number is not None:
                        actual_date = base_date + timedelta(days=day_number)
                        results.append({
                            "cosit": cosit,
                            "id": node_id,
                            "day": day_number,
                            "date": actual_date.strftime("%Y-%m-%d")
                        })

        except Exception as e:
            print(f"❌ 处理文件 {filename} 出错: {e}")

# 转换为 DataFrame
df = pd.DataFrame(results)

# 保存到 CSV 文件
output_file = "all_days_converted.csv"
df.to_csv(output_file, index=False)

print(f"✅ 所有数据已处理完毕，并保存到 {output_file}")

✅ 所有数据已处理完毕，并保存到 all_days_converted.csv


In [7]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv("all_days_converted.csv")

# 根据 cosit 和 date 进行排序
df_sorted = df.sort_values(by=["cosit", "date"])

# 保存排序后的 CSV 文件
df_sorted.to_csv("cosit_date_sorted.csv", index=False)

print(" 排序完成，已保存到 cosit_date_sorted.csv")

 排序完成，已保存到 cosit_date_sorted.csv


In [8]:
import pandas as pd
cosit_date_sorted = pd.read_csv("cosit_date_sorted.csv")
cosit_date_sorted.head()

,cosit,id,day,date
0,0000001_0101,A5C0D853-0D00-4C1A-88C4-81554113CF1C,14816,2010-07-26
1,0000001_0101,A5C0D853-0D00-4C1A-88C4-81554113CF1C,14817,2010-07-27
2,0000001_0101,A5C0D853-0D00-4C1A-88C4-81554113CF1C,14818,2010-07-28
3,0000001_0101,A5C0D853-0D00-4C1A-88C4-81554113CF1C,15201,2011-08-15
4,0000001_0101,A5C0D853-0D00-4C1A-88C4-81554113CF1C,15202,2011-08-16


## 第二种node 的爬虫
##### node=GDOT_CCS的cosit和id 即长期记录数据的（gdot网站上表示为绿色三角，cosit没下划线）点

In [1]:
import requests
import json
import pandas as pd
import os
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 读取 CSV 文件
blue_dot = pd.read_csv("cosit_and_id_for_ccs_dot.csv")

# 确保存储文件夹存在
output_dir = "ccs_dot_date_json"
os.makedirs(output_dir, exist_ok=True)

# 请求基础 URL
base_url = "https://gdottrafficdata.drakewell.com/dataserver/tabularCalendar/{}"

# 预定义多个 User-Agent
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Linux; Android 11; SM-G998B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/537.36 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
]

# 请求体
payload = {
    "startDate": "2000-01-01",
    "endDate": "2025-02-15"
}

# 进度条 & 统计信息
total_tasks = len(blue_dot)

# 爬取单个站点的函数
def fetch_data(cosit, node_id):
    """爬取指定 cosit 和 id 的数据"""
    # 生成存储文件名
    filename = os.path.join(output_dir, f"{cosit}_{node_id}.json")

    # 检查文件是否已存在，避免重复爬取
    if os.path.exists(filename):
        return f"已存在，跳过: {cosit} - {node_id}"

    # 随机选择 User-Agent
    user_agent = random.choice(USER_AGENTS)

    # 请求头
    headers = {
        "accept": "*/*",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en,zh;q=0.9,zh-CN;q=0.8",
        "cache-control": "no-cache",
        "content-type": "application/json",
        "origin": "https://gdottrafficdata.drakewell.com",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": user_agent,  # 使用随机 User-Agent
        "x-requested-with": "XMLHttpRequest",
        "cookie": "_ga=GA1.1.1478648062.1737667423; ASPSESSIONIDAWDACTBQ=FCHPNCEALJEDEOIADIMLLKPF; _ga_02J23J4R19=GS1.1.1738639325.19.1.1738639607.0.0.0"
    }

    # 更新 URL 和 Referer 头
    url = base_url.format(node_id)
    headers["referer"] = f"https://gdottrafficdata.drakewell.com/calendar_alt.asp?node=GDOT_PORTABLES&cosit={cosit}"

    try:
        # 发送请求
        response = requests.post(url, headers=headers, data=json.dumps(payload), timeout=10)

        if response.status_code == 200:
            # 解析 JSON 数据
            data = response.json()
            result = {"cosit": cosit, "id": node_id, "data": data}

            # 保存数据
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(result, f, indent=4, ensure_ascii=False)

            return f"成功获取: {cosit} - {node_id}"

        else:
            return f"请求失败: {cosit} - {node_id}, 状态码: {response.status_code}"

    except requests.exceptions.RequestException as e:
        return f"请求出错: {cosit} - {node_id}, 错误: {e}"

    finally:
        # 随机暂停 0.1 - 1.0 秒，防止封 IP
        time.sleep(random.uniform(0.1, 1.0))


# **多线程爬取**
max_workers = 10  # 可调整并发数，根据网络情况设定

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 提交任务
    futures = {executor.submit(fetch_data, row["cosit"], row["id"]): (row["cosit"], row["id"]) for _, row in blue_dot.iterrows()}

    # 进度条
    with tqdm(total=total_tasks, desc="爬取进度") as pbar:
        for future in as_completed(futures):
            result = future.result()
            print(result)  # 打印爬取状态
            pbar.update(1)  # 更新进度条

print("所有数据爬取完成！")

爬取进度:   0%|          | 1/330 [00:00<03:54,  1.40it/s]

成功获取: 150118 - E5785F82-A92B-46BE-884D-8B4DBC0508BF


爬取进度:   1%|          | 2/330 [00:01<03:10,  1.72it/s]

成功获取: 90156 - F2F2DDD4-184D-4EB9-96C3-62C1BACBECEB
成功获取: 10183 - 573292FF-2F4B-4721-92BB-B2278A482D0F
成功获取: 30138 - F61A1A60-F714-4CC5-AEA1-2C68270317F4
成功获取: 150178 - EAFBEDDA-C981-49F0-B992-38BC6481E713
成功获取: 10185 - 73997343-DF95-41E3-9001-7A7B6F3748A7


爬取进度:   3%|▎         | 9/330 [00:01<00:39,  8.05it/s]

...

成功获取: 3010196 - D1B94F8B-DA2C-4F9F-BB89-AE4D8AA2F706
成功获取: 2950035 - 5E9351A5-292B-4555-9593-7088FDD9BF47
成功获取: 2990007 - 48B5322B-12AB-4D90-8E52-0D8DC384DAB3


爬取进度:  99%|█████████▉| 326/330 [00:33<00:00,  8.72it/s]

成功获取: 2990309 - CFD95487-F373-46A4-8414-A43E6405AC24
成功获取: 2950036 - C6A65447-F569-4A62-B5B1-76DB163558D9
成功获取: 2970018 - D051E93E-D792-459B-B220-1D80CC6FDC9F
成功获取: 3190167 - CA8F5A30-D4D0-452B-A9E1-E24493D86C61


爬取进度: 100%|██████████| 330/330 [00:34<00:00,  9.69it/s]

成功获取: 3130101 - 8ADB98AB-C07B-486D-87FA-2039355EB38D
所有数据爬取完成！


In [2]:
import os
import json
import pandas as pd
from datetime import datetime, timedelta

# 设定 JSON 文件夹路径
json_folder = "ccs_dot_date_json"

# 设定起始日期（1970-01-01）
base_date = datetime(1970, 1, 1)

# 结果存储列表
results = []

# 遍历 JSON 文件夹
for filename in os.listdir(json_folder):
    if filename.endswith(".json"):
        file_path = os.path.join(json_folder, filename)

        try:
            # 读取 JSON 文件
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # 提取 cosit 和 id
            cosit = data.get("cosit", "unknown")
            node_id = data.get("id", "unknown")

            # 提取天数并转换为日期格式
            if "data" in data and "data" in data["data"]:
                for entry in data["data"]["data"]:
                    day_number = entry.get("day")
                    if day_number is not None:
                        actual_date = base_date + timedelta(days=day_number)
                        results.append({
                            "cosit": cosit,
                            "id": node_id,
                            "day": day_number,
                            "date": actual_date.strftime("%Y-%m-%d")
                        })

        except Exception as e:
            print(f"❌ 处理文件 {filename} 出错: {e}")

# 转换为 DataFrame
df = pd.DataFrame(results)

# 保存到 CSV 文件
output_file = "ccs_all_days_converted.csv"
df.to_csv(output_file, index=False)

print(f"✅ 所有数据已处理完毕，并保存到 {output_file}")

✅ 所有数据已处理完毕，并保存到 ccs_all_days_converted.csv


In [3]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv("ccs_all_days_converted.csv")

# 根据 cosit 和 date 进行排序
df_sorted = df.sort_values(by=["cosit", "date"])

# 保存排序后的 CSV 文件
df_sorted.to_csv("ccs_cosit_date_sorted.csv", index=False)

print(" 排序完成，已保存到 ccs_cosit_date_sorted.csv")

 排序完成，已保存到 ccs_cosit_date_sorted.csv


In [ ]:
# 单个网站爬虫示例

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# 读取排序后的 CSV 文件
df = pd.read_csv("all_days_sorted.csv")

# 获取所有的 cosit 和每年的起始和结束日期
grouped_data = df.groupby("cosit")["date"].agg(["min", "max"]).reset_index()

# 初始化 Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 无头模式
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

# 定义结果存储
results = []

# 遍历每个 cosit 和日期范围
for _, row in grouped_data.iterrows():
    cosit = row["cosit"]
    report_date = row["min"]  # 每年的第一个日期
    end_date = row["max"]     # 每年的最后一个日期

    # 生成目标 URL
    url = f"https://gdottrafficdata.drakewell.com/tfdaysreport.asp?node=GDOT_PORTABLES&cosit={cosit}&reportdate={report_date}&enddate={end_date}"

    try:
        # 打开页面
        driver.get(url)

        # 等待页面加载完成，直到表格出现
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "gridTable"))
        )

        # 获取表格数据
        table = driver.find_element(By.ID, "gridTable")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 提取表格内容
        for row in rows:
            cells = row.find_elements(By.TAG_NAME, "td")
            if cells:  # 确保不是空行
                data = [cell.text for cell in cells]
                results.append([cosit, report_date, end_date] + data)

        print(f"✅ 成功爬取 cosit: {cosit} ({report_date} - {end_date})")

    except Exception as e:
        print(f"❌ 爬取失败 cosit: {cosit} ({report_date} - {end_date}), 错误: {e}")

    # 随机暂停一段时间，避免被封 IP
    time.sleep(2)

# 关闭浏览器
driver.quit()

# 将结果保存为 CSV 文件
output_file = "cosit_table_data.csv"
columns = ["cosit", "report_date", "end_date", "hour", "value1", "value2", "value3"]
df_results = pd.DataFrame(results, columns=columns)
df_results.to_csv(output_file, index=False)

print(f"✅ 所有数据已爬取完毕，并保存到 {output_file}")


❌ 爬取失败 cosit: 0000001_0101 (2010-07-26 - 2021-01-20), 错误: Message: 
Stacktrace:
0   chromedriver                        0x00000001026bf0d4 cxxbridge1$str$ptr + 2600792
1   chromedriver                        0x00000001026b79f0 cxxbridge1$str$ptr + 2570356
2   chromedriver                        0x00000001022583d8 cxxbridge1$string$len + 89376
3   chromedriver                        0x000000010229d414 cxxbridge1$string$len + 372060
4   chromedriver                        0x00000001022d7638 cxxbridge1$string$len + 610176
5   chromedriver                        0x000000010229134c cxxbridge1$string$len + 322708
6   chromedriver                        0x0000000102291f94 cxxbridge1$string$len + 325852
7   chromedriver                        0x000000010268a238 cxxbridge1$str$ptr + 2384060
8   chromedriver                        0x000000010268d550 cxxbridge1$str$ptr + 2397140
9   chromedriver                        0x0000000102670b9c cxxbridge1$str$ptr + 2279968
10  chromedriver               

KeyboardInterrupt: 

In [13]:
from selenium import webdriver
import time
import random
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# 设置选项
options = Options()
#options.add_argument("--headless")  # 无头模式
#options.add_argument("--disable-blink-features=AutomationControlled")  # 绕过反爬

# 启动浏览器
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 访问目标网站
url = "https://gdottrafficdata.drakewell.com/tfdaysreport.asp?node=GDOT_PORTABLES&cosit=0000001_0101&reportdate=2010-07-26&enddate=2010-07-28"
driver.get(url)

# 注入 Cookies
cookies = {
    "ASPSessionIDAWDACTBQ": "CJIJNCEAKCABKPAIEBPJHPCE",
    "ASPSessionIDCWCDBQCT": "NNFDKOABKFMMGJPLDNCPKHOE",
    "_ga": "GA1.1.147648062.1737667423",
    "_ga_02J23J4R19": "GS1.1.1738683193.21.0.1738683195.0.0"
}

for name, value in cookies.items():
    driver.add_cookie({"name": name, "value": value, "domain": "gdottrafficdata.drakewell.com"})

# 刷新页面以加载内容
driver.refresh()
time.sleep(random.uniform(1, 3))

# 获取页面源码
html = driver.page_source
print(html)

# 关闭浏览器
#driver.quit()

<html lang="en"><head>    <meta charset="utf-8">    <title>Error</title>
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="viewport" content="width=device-width, initial-scale=1">
		<link rel="icon" href="favicon.ico">
		
		<!-- Analytics -->
		<!-- Global Site Tag (gtag.js) - Google Analytics -->
	<script async="" type="text/javascript" data-cookieconsent="statistics" cookie-consent="tracking" src="https://www.googletagmanager.com/gtag/js?id=G-02J23J4R19"></script>
	<script type="text/javascript" data-cookieconsent="statistics">
	  window.dataLayer = window.dataLayer || [];
	  function gtag(){dataLayer.push(arguments)};
	  gtag('js', new Date());
	  gtag('config', 'G-02J23J4R19');
	</script>


		<!-- shims -->
		
		
		<!-- jQuery -->
		
	<script type="text/javascript" src="scripts/libs/jquery/jquery-3.5.1.min.js"></script>
	

		<!-- jQuery-UI -->
		
	<link rel="stylesheet" type="text/css" href="libs/jquery-ui/jquery-ui.min.css">
	<script type="text/javascript" src=